## Conversion to TensorFlow Records

In [1]:
import os
import glob
import pandas as pd
import io
import xml.etree.ElementTree as ET
import argparse

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow.compat.v1 as tf
from PIL import Image
from object_detection.utils import dataset_util, label_map_util
from collections import namedtuple

In [2]:
from types import SimpleNamespace

To use this notebook, change `dataset_type` to be one of the following: train, eval, test. Once you have done that, you can execute the other cells to generate the tf_records.

In [24]:
args = SimpleNamespace()
dataset_type = 'train'
args.xml_dir = f'../../vbd_vol/tf_obj_files/{dataset_type}_labels'
args.image_dir = f'../../vbd_vol/tf_obj_files/{dataset_type}'
args.csv_path = f'../../vbd_vol/tf_obj_files/{dataset_type}_table.csv'
args.labels_path = '../../vbd_vol/tf_obj_files/label_map.pbtxt'
args.output_path = f'../../vbd_vol/tf_obj_files/{dataset_type}.record'

In [25]:
label_map = label_map_util.load_labelmap(args.labels_path)
label_map_dict = label_map_util.get_label_map_dict(label_map)

In [26]:
def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(float(member[5][0].text)), # xmin
                     int(float(member[5][1].text)), # xmax
                     int(float(member[5][2].text)), # ymin
                     int(float(member[5][3].text))  # ymax
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'xmax', 'ymin', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [27]:
def class_text_to_int(row_label):
    return label_map_dict[row_label]

In [28]:
def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x 
            in zip(gb.groups.keys(), gb.groups)]

In [29]:
def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        assert width == 1024 and height == 1024, f"width={width}, height={height}"
        
        assert row['xmin'] >= 0, f"Weird xmin: index={index}, img={row['filename']}"
        assert row['ymin'] >= 0, f"Weird ymin: index={index}, img={row['filename']}"
        assert row['xmax'] >= 0, f"Weird xmax: index={index}, img={row['filename']}"
        assert row['ymax'] >= 0, f"Weird ymax: index={index}, img={row['filename']}"
        
        assert row['xmin'] / width <= 1, f"oob xmin: index={index}, img={row['filename']}"
        assert row['ymin'] / height <= 1, f"oob ymin: index={index}, img={row['filename']}"
        assert row['xmax'] / width <= 1, f"oob xmax: index={index}, img={row['filename']}"
        assert row['ymax'] / height <= 1, f"oob ymax: index={index}, img={row['filename']}"
        
        assert row['xmin'] <= row['xmax'], f"xmin > xmax: index={index}, img={row['filename']}"
        assert row['ymin'] <= row['ymax'], f"ymin > ymax: index={index}, img={row['filename']}"
        
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    
    # print(tf_example)
    return tf_example

In [30]:
writer = tf.python_io.TFRecordWriter(args.output_path)
path = os.path.join(args.image_dir)
examples = xml_to_csv(args.xml_dir)
grouped = split(examples, 'filename')
for group in grouped:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())
writer.close()
print('Successfully created the TFRecord file: {}'.format(args.output_path))
if args.csv_path is not None:
    examples.to_csv(args.csv_path, index=None)
    print('Successfully created the CSV file: {}'.format(args.csv_path))

Successfully created the TFRecord file: ../../vbd_vol/tf_obj_files/train.record
Successfully created the CSV file: ../../vbd_vol/tf_obj_files/train_table.csv
